In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv("atp_tennis.csv")


In [9]:
df.head()

,Tournament,Date,Series,Court,Surface,Round,Best of,Player_1,Player_2,Winner,Rank_1,Rank_2,Pts_1,Pts_2,Odd_1,Odd_2,Score
0,Australian Hardcourt Championships,2000-01-03,International,Outdoor,Hard,1st Round,3,Dosedel S.,Ljubicic I.,Dosedel S.,63,77,-1,-1,-1.0,-1.0,6-4 6-2
1,Australian Hardcourt Championships,2000-01-03,International,Outdoor,Hard,1st Round,3,Clement A.,Enqvist T.,Enqvist T.,56,5,-1,-1,-1.0,-1.0,3-6 3-6
2,Australian Hardcourt Championships,2000-01-03,International,Outdoor,Hard,1st Round,3,Escude N.,Baccanello P.,Escude N.,40,655,-1,-1,-1.0,-1.0,6-7 7-5 6-3
3,Australian Hardcourt Championships,2000-01-03,International,Outdoor,Hard,1st Round,3,Knippschild J.,Federer R.,Federer R.,87,65,-1,-1,-1.0,-1.0,1-6 4-6
4,Australian Hardcourt Championships,2000-01-03,International,Outdoor,Hard,1st Round,3,Fromberg R.,Woodbridge T.,Fromberg R.,81,198,-1,-1,-1.0,-1.0,7-6 5-7 6-4


In [10]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder


In [17]:
encoder = OneHotEncoder(sparse_output=False)
encoded_court = encoder.fit_transform(df[['Court']])

In [20]:
df["Series"].unique()

array(['International', 'Grand Slam', 'International Gold', 'Masters',
       'Masters Cup', 'ATP250', 'ATP500', 'Masters 1000'], dtype=object)

In [23]:
df["Date"] = pd.to_datetime(df["Date"])

In [24]:
df_new = df[df["Date"] > "2010-01-01"]

In [26]:
df_new

,Tournament,Date,Series,Court,Surface,Round,Best of,Player_1,Player_2,Winner,Rank_1,Rank_2,Pts_1,Pts_2,Odd_1,Odd_2,Score
27525,Brisbane International,2010-01-04,ATP250,Outdoor,Hard,1st Round,3,Nieminen J.,Gasquet R.,Gasquet R.,88,52,568,850,2.62,1.44,3-6 6-4 4-6
27526,Brisbane International,2010-01-04,ATP250,Outdoor,Hard,1st Round,3,Odesnik W.,Clement A.,Odesnik W.,105,63,521,667,2.25,1.57,6-4 7-6
27527,Brisbane International,2010-01-04,ATP250,Outdoor,Hard,1st Round,3,Petzschner P.,Gicquel M.,Gicquel M.,80,58,587,744,2.20,1.61,3-6 1-6
27528,Brisbane International,2010-01-04,ATP250,Outdoor,Hard,1st Round,3,Falla A.,Chardy J.,Falla A.,81,32,587,1102,2.62,1.44,6-4 6-4
27529,Brisbane International,2010-01-04,ATP250,Outdoor,Hard,1st Round,3,Llodra M.,Levy H.,Levy H.,67,119,649,461,1.36,3.00,2-6 0-6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66676,Masters Cup,2025-11-14,Masters Cup,Indoor,Hard,Round Robin,3,Sinner J.,Shelton B.,Sinner J.,2,5,10000,3970,1.05,10.00,6-3 7-6
66677,Masters Cup,2025-11-14,Masters Cup,Indoor,Hard,Round Robin,3,Zverev A.,Auger-Aliassime F.,Auger-Aliassime F.,3,8,4960,3845,4.50,1.20,4-6 6-7
66678,Masters Cup,2025-11-15,Masters Cup,Indoor,Hard,Semifinals,3,Sinner J.,De Minaur A.,Sinner J.,2,7,10000,3935,1.05,10.00,7-5 6-2
66679,Masters Cup,2025-11-15,Masters Cup,Indoor,Hard,Semifinals,3,Auger-Aliassime F.,Alcaraz C.,Alcaraz C.,8,1,3845,11050,4.50,1.20,2-6 4-6


In [27]:
df.columns

Index(['Tournament', 'Date', 'Series', 'Court', 'Surface', 'Round', 'Best of',
       'Player_1', 'Player_2', 'Winner', 'Rank_1', 'Rank_2', 'Pts_1', 'Pts_2',
       'Odd_1', 'Odd_2', 'Score'],
      dtype='object')

In [29]:
sparse = []
new_cols = []
numeric_cols = []

for col in df.columns:
    
    if df[col].dtype == "object":
        enc = OneHotEncoder(sparse_output=True, handle_unknown="ignore")
        encoded_col = enc.fit_transform(df[[col]])
        sparse.append(encoded_col)
        new_cols.extend(enc.get_feature_names_out([col]))
    else:
        numeric_cols.append(df[[col]])
        new_cols.append(col)
        

In [31]:
from scipy.sparse import hstack

In [32]:
if sparse:
    sparse_all = hstack(sparse)
else:
    sparse_all = None


In [33]:
from scipy.sparse import csr_matrix

In [37]:
for col in numeric_cols.columns:
    if numeric_cols[col].apply(lambda x: isinstance(x, (list, np.ndarray))).any():
        print(f"Column '{col}' contains arrays/lists!")

AttributeError: 'list' object has no attribute 'columns'

In [39]:
if sparse is not None and numeric_cols is not None:
    # Convert list to DataFrame first, then to float and sparse
    numeric_df = pd.DataFrame(numeric_cols)
    numeric_sparse = csr_matrix(numeric_df.astype(float))
    X_sparse = hstack([sparse, numeric_sparse])
elif sparse is not None:
    X_sparse = sparse
else:
    numeric_df = pd.DataFrame(numeric_cols)
    numeric_sparse = csr_matrix(numeric_df.astype(float))
    X_sparse = numeric_sparse

ValueError: Must pass 2-d input. shape=(8, 66681, 1)